In [ ]:
from keras.applications.inception_v3 import InceptionV3

from utils import *
path = './data/'

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

## Prep Data

In [ ]:
#Now for future iterations, just load the array
val_batches = get_batches(path + 'valid/', batch_size=32, target_size=(299,299))
tst_batches = get_batches(path + 'test_stg1/', batch_size=32, target_size=(299,299))

aug_gen = ImageDataGenerator(rotation_range=15, 
                                   width_shift_range=.2, 
                                   height_shift_range=.2, 
                                   zoom_range=.5,
                                  channel_shift_range=.5,
                                  horizontal_flip=True,
                                  vertical_flip=True)

tr_batches = get_batches(path + 'train/', batch_size=32, 
                         gen=aug_gen, target_size=(299,299))


## Fine Tune the Inception V3 Model 
- [Source Code for Inception](https://github.com/fchollet/keras/blob/master/keras/applications/inception_v3.py)
- Skipping the final layer
- Make the layers untrainable
- Add in the fully connected block again
- Train for a few epochs
- Eval

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

#Fully connected blocks + Dropout
x = Dense(1024, activation='relu')(x)
x = Dropout(.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(.5)(x)
predictions = Dense(8, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])



In [ ]:
fname = 'models/inceptionV3_0/weights.{epoch:02d}-{val_loss:.2f}.h5'
callbacks = [ModelCheckpoint(path + fname,)]
#model.fit_generator(tr_batches, tr_batches.nb_sample * 3, nb_epoch=5,
#                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample,
#                    callbacks=callbacks)

## Add Some Psuedo Labeling
- Load best weights
- Create Predictions for the test set
-Create batches for test, valid and train

In [ ]:
model_path = './data/models/inceptionV3_0/'
model.load_weights(model_path + 'weights.04-0.75.h5')

In [ ]:
test = get_data(path + 'test_stg1/', target_size=(299,299))
preds = model.predict(test, batch_size=64)

In [ ]:
#Vanilla generator & Augmentation Generator
gen = ImageDataGenerator()
aug_gen = ImageDataGenerator(rotation_range=15, 
                                   width_shift_range=.2, 
                                   height_shift_range=.2, 
                                   zoom_range=.5,
                                  channel_shift_range=.5,
                                  horizontal_flip=True,
                                  vertical_flip=True)

tr_batches = get_batches(path + 'train/', batch_size=44, 
                         gen=aug_gen, target_size=(299,299))
val_batches = get_batches(path + 'valid/', batch_size=8, target_size=(299,299))
test_baches = gen.flow(test, preds, batch_size=16)

In [ ]:
#Creates a way to mix up the train, test, validation data (44, 16, 8 images per batch)
mi = MixIterator([tr_batches, val_batches, test_baches])

In [ ]:
fname = 'models/inceptionV3_0/weights.pseudolabeling.{epoch:02d}-{val_loss:.2f}.h5'
callbacks = [ModelCheckpoint(path + fname)]
model.fit_generator(mi, mi.N, nb_epoch=5, callbacks=callbacks,
                    validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

## Try Ensembling to Generate a prediction
- Generate multiple predictions for the same test images with minor variations

In [ ]:
#Settings
nb_test_samples = 1000
nb_classes = 8
nb_runs = 5
nb_aug = 5
arch = 'Inception'

#Load best weights
model_path = './data/models/inceptionV3_0/'
model.load_weights(model_path + 'weights.04-0.75.h5')

def predict_test(aug=False, img_size=(299,299)):
    if aug:
        print "Using data augmentation"
        test_gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
                              channel_shift_range=10, height_shift_range=0.05, shear_range=0.05,
                             horizontal_flip=True)
    else:
        test_gen = ImageDataGenerator()
    
    #Image generator with/without aug
    test_data = test_gen.flow_from_directory(path + 'test_stg1/', shuffle=False, 
                                            batch_size=32, target_size=img_size)
    preds = model.predict_generator(test_data, test_data.nb_sample)
    filenames = test_data.filenames
    return preds, filenames


def ensemble_predictions():    
    predictions_full = np.zeros((nb_test_samples, nb_classes))
    
    for run in range(nb_runs):
        print("\nStarting Prediction Run {0} of {1}...\n".format(run+1, nb_runs))
        predictions_aug = np.zeros((nb_test_samples, nb_classes))
        
        for aug in range(nb_aug):
            print("\n--Predicting on Augmentation {0} of {1}...\n".format(aug+1, nb_aug))          
            print("----Predicting on {} model...".format(arch))
            pred, filenames = predict_test(aug=aug)
            predictions_aug += pred
        
        #Avg all augmentation runs
        predictions_aug /= nb_aug
        predictions_full += predictions_aug
    
    #Avg all runs 
    predictions_full /= nb_runs
    return predictions_full, filenames

predictions, filenames = ensemble_predictions()

## Generate Submission

In [ ]:
from nc_utils import *
preds_clip = do_clip(predictions,.99)
img_ids = [img.split('/')[1] for img in filenames]
results, fname = create_submission_file(preds_clip, img_ids, "inceptionV3_0_ensembling")
comp = 'the-nature-conservancy-fisheries-monitoring'
info = 'inceptionV3_0 + Data Aug + Ensembling 99 clip'
cmd = gen_submission_cmd(fname, comp, info)
print(cmd)